In [2]:
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np

from tensorflow.keras.optimizers import Adam

tfd = tfp.distributions


In [4]:
# Define the inputs for emissions and transitions
time_steps = 100  # Example number of time steps
num_states = 3    # Number of latent states
dim_inputs_emission = 2

# Input-driven emissions (external inputs u_emm_t for emission)
u_emm = tf.random.uniform([time_steps, dim_inputs_emission])

# Input-driven transitions (external inputs u_tr_t for transition)
dim_inputs_transition = 2
u_tr = tf.random.uniform([time_steps, dim_inputs_transition])

# Define model parameters for emission and transition
mu_emission = tf.Variable(tf.random.normal([num_states, dim_inputs_emission]), name='mu_emission')
kappa_emission = tf.Variable(tf.ones([num_states]), name='kappa_emission')

A_transition = tf.Variable(tf.random.normal([num_states, num_states]), name='A_transition')
B_transition = tf.Variable(tf.random.normal([num_states, dim_inputs_transition]), name='B_transition')

# Define initial state prior
initial_state_logits = tf.Variable(tf.zeros([num_states]), name='initial_state_logits')

In [5]:

def emission_distribution_fn(state, u_emm_t):
    """
    Creates a von Mises emission distribution for the given state, using u_emm_t to modify the mean.
    """
    mu = tf.reduce_sum(mu_emission[state] * u_emm_t)  # Custom emission component: mean driven by input
    kappa = kappa_emission[state]
    return tfd.VonMises(loc=mu, concentration=kappa)

def transition_distribution_fn(state, u_tr_t):
    """
    Creates a transition distribution given the current state and the input u_tr_t.
    """
    logits = tf.reduce_sum(A_transition[state] * state + u_tr_t @ tf.transpose(B_transition))
    return tfd.Categorical(logits=logits)

# Build the HMM
def make_hmm():
    return tfd.HiddenMarkovModel(
        initial_distribution=tfd.Categorical(logits=initial_state_logits),
        transition_distribution=lambda state: transition_distribution_fn(state, u_tr[state]),
        observation_distribution=lambda state: emission_distribution_fn(state, u_emm[state]),
        num_steps=time_steps
    )

# Create the HMM
hmm = make_hmm()

AttributeError: 'function' object has no attribute 'event_shape'

In [ ]:
# Define target data (simulated observations for training)
target_data = tf.random.uniform([time_steps], maxval=num_states, dtype=tf.int32)

# Define the loss function
def neg_log_likelihood():
    return -tf.reduce_mean(hmm.log_prob(target_data))

# Define the optimizer
optimizer = Adam(learning_rate=0.01)

# Training step
@tf.function
def train_step():
    with tf.GradientTape() as tape:
        loss = neg_log_likelihood()
    gradients = tape.gradient(loss, [initial_state_logits, mu_emission, kappa_emission, A_transition, B_transition])
    optimizer.apply_gradients(zip(gradients, [initial_state_logits, mu_emission, kappa_emission, A_transition, B_transition]))
    return loss


In [ ]:
# Train the model
num_epochs = 500
for epoch in range(num_epochs):
    loss = train_step()
    if epoch % 50 == 0:
        print(f"Epoch {epoch}, Loss: {loss.numpy()}")

# Sample from the HMM or compute log probabilities
samples = hmm.sample()
log_prob = hmm.log_prob(samples)

print("Samples:", samples)
print("Log Probability:", log_prob)